In [5]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
data.head()





,Text,Emotions
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [6]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [7]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

In [8]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
400/400 [==============================] - 17s 41ms/step - loss: 1.3490 - accuracy: 0.4825 - val_loss: 0.8827 - val_accuracy: 0.6984
Epoch 2/10
400/400 [==============================] - 15s 37ms/step - loss: 0.3640 - accuracy: 0.8851 - val_loss: 0.5515 - val_accuracy: 0.8150
Epoch 3/10
400/400 [==============================] - 15s 38ms/step - loss: 0.0630 - accuracy: 0.9835 - val_loss: 0.5638 - val_accuracy: 0.8259
Epoch 4/10
400/400 [==============================] - 15s 38ms/step - loss: 0.0252 - accuracy: 0.9947 - val_loss: 0.6143 - val_accuracy: 0.8194
Epoch 5/10
400/400 [==============================] - 15s 37ms/step - loss: 0.0186 - accuracy: 0.9964 - val_loss: 0.6395 - val_accuracy: 0.8253
Epoch 6/10
400/400 [==============================] - 19s 47ms/step - loss: 0.0170 - accuracy: 0.9962 - val_loss: 0.6933 - val_accuracy: 0.8163
Epoch 7/10
400/400 [==============================] - 18s 45ms/step - loss: 0.0152 - accuracy: 0.9970 - val_loss: 0.6877 - val_accuracy:

In [9]:
model_architecture = model.to_json()
with open("model_architecture3.json", "w") as json_file:
    json_file.write(model_architecture)

model.save_weights("model_weights3.h5")

In [12]:
input_text = "I'm smart and talented and open to new experiences"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 37ms/step
['joy']
